# Mean and Eddy vorticity fluxes

In [1]:
import os
import sys
sys.path.append('../..')
import glob
import time
import xarray
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cmocean
from numba import float64, guvectorize
import cimf as c
from ddxddy import ddxND, ddyND
%load_ext autoreload
%autoreload 2

plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['Helvetica']
plt.rcParams['font.cursive'] = ['Helvetica-Oblique']
plt.rcParams['pdf.fonttype'] = 42

In [2]:
p_ref = 100000
cp = 1004
Rdry = 286.9
kappa = Rdry/cp

In [3]:
dataloc = "/Users/jasperdejong/Documents/PhD/Irma/Data/LambertGrid/629x989interped/"
#dataloc2 = "/Users/jasperdejong/Documents/PhD/Irma/Data/LambertGrid/629x989/"
files = sorted(glob.glob(dataloc+'*.nc'))
#files2 = sorted(glob.glob(dataloc2+'*.nc'))

In [4]:
dst = xarray.open_mfdataset(files, combine='nested', concat_dim='valid_time')

In [5]:
dtdp = dst.theta.diff('theta') / dst.pres.diff('theta')

In [6]:
%%time
ds = xarray.open_mfdataset(files, combine='nested', concat_dim='valid_time')
ds = ds.drop_vars(['q','tke','height','Ke','dtdp'])
ds = c.crop(ds)
ds = ds.swap_dims(theta='z')
ds['cimf'] = c.calc_cimf(ds)
dvdx = ddxND(ds.v, ds.latitude, ds.longitude)
dudy = ddyND(ds.u, ds.latitude, ds.longitude)
av = (dvdx - dudy + c.fc).transpose(*ds.u.dims)
av = xarray.DataArray(av, coords=ds.u.coords, dims=ds.u.dims, name='eta')
av.attrs = {'long_name':'absolute vorticity', 'units':'s**-1'}
ds['eta'] = av
dp = c.calc_dp(ds)
dth = ds.theta.differentiate('z')
sigma = -1/c.g*dp/dth
ds['dthetadt'] = ds.cimf/sigma
ds = c.convert_wind(ds)
ds['dvdtheta'] = ds.v_tan.differentiate('theta')

08:08:49: crop()
08:09:08: calc_cimf()
08:09:08: masscontinuity(lagrange2d)
08:09:08: latlon2dxdy()
08:09:08: translational_velocity()
08:09:08: calc_dp(ds)
08:09:09: masstendency()
08:09:10: iwind2
08:09:12: FX, FY in masscontinuity()
08:09:30: ddx, ddy in masscontinuity()
08:09:54: integrate_dcimf()
08:10:07: calc_dp(ds)
08:10:12: convert_wind()
08:10:12: toPolar()
08:10:12: distance()
08:10:12: translational_velocity()
08:10:17: transform_vector()
CPU times: user 53 s, sys: 1min 6s, total: 1min 59s
Wall time: 2min 2s


In [7]:
dss = ds[['latitude','longitude','xc','yc','u_rad','v_tan','eta','dthetadt','dvdtheta','pres']]
dp = c.calc_dp(ds)
dth = ds.theta.differentiate('z')
dss['sigma'] = -1/c.g*dp/dth
dss['dthetadt'] = ds.cimf/dss.sigma

08:10:49: calc_dp(ds)


In [8]:
%%time
dsm,_ = c.azimean(dss)

08:10:59: azimean()
08:10:59: toPolar()
08:10:59: distance()
08:11:01: convert_wind()
08:11:01: convert_wind(): using existing u_rad and v_tan
08:11:01: azimean_gufunc()
08:11:35: removing ['dy', 'dx'] from azimuthal mean dataset
CPU times: user 14.7 s, sys: 12 s, total: 26.7 s
Wall time: 35.5 s


In [9]:
dsm['Ja_m'] = dsm.u_rad * dsm.eta
dsm['Jh_m'] = dsm.dthetadt * dsm.dvdtheta

In [10]:
# convert azimuthal means back to original grid
dist, angle = c.toPolar(ds.latitude, ds.longitude, ds.xc, ds.yc)
dist.attrs = {'long_name':'radial distance from centre','units':'m'}
ds['radius'] = dist.compute().astype('float32')
u_rad_m = dsm.u_rad.interp(r=ds.radius)
eta_m = dsm.eta.interp(r=ds.radius)
dthetadt_m = dsm.dthetadt.interp(r=ds.radius)
dvdtheta_m = dsm.dvdtheta.interp(r=ds.radius)

08:11:35: toPolar()
08:11:35: distance()


In [11]:
# calculate eddy fluxes
u_e = ds['u_rad'] - u_rad_m
eta_e = ds['eta'] - eta_m 
dthetadt_e = ds['dthetadt'] - dthetadt_m 
dvdtheta_e = ds['dvdtheta'] - dvdtheta_m
ds['Ja_e'] = u_e * eta_e
ds['Jh_e'] = dthetadt_e * dvdtheta_e

In [12]:
# azimuthal mean of eddy components
dsme,_ = c.azimean(ds[['latitude','longitude','xc','yc','Ja_e','Jh_e']])

08:13:56: azimean()
08:13:56: toPolar()
08:13:56: distance()
08:13:57: convert_wind()
08:13:57: convert_wind(): no input, skipping
08:13:57: azimean_gufunc()
08:14:04: removing ['dy', 'dx'] from azimuthal mean dataset


In [13]:
dsm = xarray.merge((dsm, dsme))
dsm = dsm.sel(valid_time=slice('2017-09-06 00','2017-09-06 12')).mean('valid_time')

In [15]:
dsm.to_netcdf('lg2meaneddy.nc')